Adhesion Demo
==============

This example demonstrates how to specify cell adhesion on the basis of molecular species.

Adjust the sliders to change the binding energy of two molecules, each of which is only on the surface of a particular
cell type. A third cell type has neither molecule.

Basic setup
------------
An interactive CC3D simulation can be initialized from a list of core specs.
Start a list of core specs that define the simulation by specifying a two-dimensional simulation
with a 100x100 lattice and second-order Potts neighborhood, and metadata to use multithreading

In [ ]:
from cc3d.core.PyCoreSpecs import Metadata, PottsCore

dim_x = dim_y = 100
specs = [
    Metadata(num_processors=4),
    PottsCore(dim_x=dim_x,
              dim_y=dim_y,
              neighbor_order=2,
              boundary_x="Periodic",
              boundary_y="Periodic")
]

Cell Types
-----------

Define three cell types called "T1" through "T3".

In [ ]:
from cc3d.core.PyCoreSpecs import CellTypePlugin

cell_types = ["T1", "T2", "T3"]
specs.append(CellTypePlugin(*cell_types))

Volume Constraint
------------------

Assign a volume constraint to all cell types.

In [ ]:
from cc3d.core.PyCoreSpecs import VolumePlugin

volume_specs = VolumePlugin()
for ct in cell_types:
    volume_specs.param_new(ct, target_volume=25, lambda_volume=2)
specs.append(volume_specs)

Adhesion
---------

Assign uniform adhesion to all cells, and additional adhesion by molecular species.

In [ ]:
from cc3d.core.PyCoreSpecs import ContactPlugin, AdhesionFlexPlugin

contact_specs = ContactPlugin(neighbor_order=2)
for idx1 in range(len(cell_types)):
    contact_specs.param_new(type_1="Medium", type_2=cell_types[idx1], energy=16)
    for idx2 in range(idx1, len(cell_types)):
        contact_specs.param_new(type_1=cell_types[idx1], type_2=cell_types[idx2], energy=16)
specs.append(contact_specs)

adhesion_specs = AdhesionFlexPlugin(neighbor_order=2)
adhesion_specs.density_new(molecule="M1", cell_type="T1", density=1.0)
adhesion_specs.density_new(molecule="M2", cell_type="T2", density=1.0)
formula = adhesion_specs.formula_new(formula_name='Binary')
formula.param_set("M1", "M1", -10.0)
formula.param_set("M1", "M2", 0.0)
formula.param_set("M2", "M2", 10.0)
specs.append(adhesion_specs)

Cell Distribution Initialization
---------------------------------

Initialize cells over the entire domain.

In [ ]:
from cc3d.core.PyCoreSpecs import UniformInitializer

unif_init_specs = UniformInitializer()
unif_init_specs.region_new(width=5, pt_min=(0, 0, 0), pt_max=(dim_x, dim_y, 1),
                           cell_types=["T1", "T1", "T2", "T2", "T3"])
specs.append(unif_init_specs)

Simulation Launch
------------------

Initialize a CC3D simulation service instance and register all simulation specification.

In [ ]:
from cc3d.CompuCellSetup.CC3DCaller import CC3DSimService

cc3d_sim = CC3DSimService()
cc3d_sim.register_specs(specs)
cc3d_sim.run()
cc3d_sim.init()
cc3d_sim.start()

Steering
---------

Add sliders to adjust molecular adhesion model parameters during simulation execution.

In [ ]:
import ipywidgets

def _cb_m1_m1(change):
    if change['name'] == 'value':
        adhesion_specs.formula['Binary']['M1']['M1'] = change.new
        adhesion_specs.steer()

def _cb_m1_m2(change):
    if change['name'] == 'value':
        adhesion_specs.formula['Binary']['M1']['M1'] = change.new
        adhesion_specs.steer()

def _cb_m2_m2(change):
    if change['name'] == 'value':
        adhesion_specs.formula['Binary']['M2']['M2'] = change.new
        adhesion_specs.steer()

slider_m1_m1 = ipywidgets.FloatSlider(
    value=adhesion_specs.formula['Binary']['M1']['M1'], 
    min=-10, 
    max=10, 
    step=0.1, 
    continuous_update=False, 
    description='M1-M1'
)
slider_m1_m2 = ipywidgets.FloatSlider(
    value=adhesion_specs.formula['Binary']['M1']['M2'], 
    min=-10, 
    max=10, 
    step=0.1, 
    continuous_update=False, 
    description='M1-M2'
)
slider_m2_m2 = ipywidgets.FloatSlider(
    value=adhesion_specs.formula['Binary']['M2']['M2'], 
    min=-10, 
    max=10, 
    step=0.1, 
    continuous_update=False, 
    description='M2-M2'
)

slider_m1_m1.observe(_cb_m1_m1, names='value')
slider_m1_m2.observe(_cb_m1_m2, names='value')
slider_m2_m2.observe(_cb_m2_m2, names='value')

Visualization
--------------

Show a single frame to visualize simulation data as it is generated and all steering widgets.

In [ ]:
from IPython.display import display

cc3d_sim.visualize(plot_freq=10).show()
display(slider_m1_m1)
display(slider_m1_m2)
display(slider_m2_m2)
display(cc3d_sim.jupyter_run_button())